## Day 7: Scheduler & Automation Pipeline Test

<img style="float: right;" src="../img/logo.png" width="120"><br>

<div style="text-align: right"> <b>Research Curator Team</b></div>
<div style="text-align: right"> Initial issue : 2025.12.04 </div>
<div style="text-align: right"> last update : 2025.12.04 </div>

개정 이력  
- `2025.12.04` : Day 7 통합 테스트 (4개 Checkpoint)

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv()
print("✅ Setup complete")

## Overview

Day 7에서는 4개의 Checkpoint를 통해 전체 자동화 파이프라인을 구축하고 테스트합니다:

1. **Checkpoint 1**: Database Models & CRUD Operations
2. **Checkpoint 2**: Scheduler & Scheduled Tasks
3. **Checkpoint 3**: Full Pipeline Integration
4. **Checkpoint 4**: Scheduler API & Monitoring

### Full Pipeline Architecture

```
┌─────────────────┐
│  1. Collect     │  ← ArXiv, News APIs (01:00 KST)
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  2. Process     │  ← LLM (Summarize, Score, Classify) (01:30 KST)
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  3. Store       │  ← PostgreSQL + Qdrant
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  4. Curate      │  ← Select top N articles
└────────┬────────┘
         │
         ▼
┌─────────────────┐
│  5. Send Email  │  ← SMTP (08:00 KST)
└─────────────────┘
         │
         ▼
┌─────────────────┐
│  6. Monitor     │  ← REST API
└─────────────────┘
```

---

# Checkpoint 1: Database Models & CRUD Operations

데이터베이스 모델의 관계 설정 및 CRUD 작업을 테스트합니다.

### 1.1 Database Connection Test

In [ ]:
from app.core.config import settings
from app.db.session import SessionLocal, engine

print(f"Database URL: {settings.database_url_str}")
print(f"Engine: {engine}")

# Test connection
try:
    with engine.connect() as conn:
        print("✅ Database connection successful!")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

### 1.2 User CRUD Operations

In [ ]:
from app.db import crud
from app.db.session import SessionLocal

db = SessionLocal()

# Create user
test_user = crud.create_user(
    db=db,
    email="test@example.com",
    name="Test User"
)

print(f"✅ Created user: {test_user.email}")
print(f"   User ID: {test_user.id}")

user_id = test_user.id

# Read user
user = crud.get_user_by_email(db, "test@example.com")
print(f"✅ Retrieved user: {user.name}")

# Update user
updated_user = crud.update_user(db, user_id, name="Updated Test User")
print(f"✅ Updated user name: {updated_user.name}")

### 1.3 UserPreference CRUD Operations

In [ ]:
# Create preferences
preference = crud.create_user_preference(
    db=db,
    user_id=user_id,
    research_fields=["Machine Learning", "NLP"],
    keywords=["transformers", "attention", "LLM"],
    sources={"arxiv": True, "google_scholar": True},
    info_types={"paper": 50, "news": 30, "report": 20},
    email_time="09:00",
    daily_limit=10,
)

print(f"✅ Created preference")
print(f"   Research fields: {preference.research_fields}")
print(f"   Keywords: {preference.keywords}")

# Update preferences
updated_pref = crud.update_user_preference(
    db, user_id,
    keywords=["transformers", "attention", "LLM", "GPT"],
    daily_limit=15,
)
print(f"✅ Updated preference")
print(f"   New keywords: {updated_pref.keywords}")
print(f"   New daily limit: {updated_pref.daily_limit}")

### 1.4 CollectedArticle CRUD Operations

In [ ]:
from datetime import datetime, UTC

# Create articles
article1 = crud.create_article(
    db=db,
    title="Attention Is All You Need",
    content="Full paper content here...",
    summary="혁신적인 Transformer 아키텍처를 소개하는 논문입니다.",
    source_url="https://arxiv.org/abs/1706.03762",
    source_type="paper",
    category="Deep Learning",
    importance_score=0.95,
    article_metadata={"authors": ["Vaswani et al."], "citations": 90000},
    published_at=datetime.now(UTC),
)

article2 = crud.create_article(
    db=db,
    title="GPT-4 Technical Report",
    content="Technical report content...",
    summary="OpenAI의 최신 대규모 언어 모델에 대한 기술 리포트입니다.",
    source_url="https://arxiv.org/abs/2303.08774",
    source_type="paper",
    category="LLM",
    importance_score=0.98,
)

article3 = crud.create_article(
    db=db,
    title="AI News: New Breakthrough",
    content="News article content...",
    source_url="https://techcrunch.com/ai-breakthrough",
    source_type="news",
    importance_score=0.75,
)

print(f"✅ Created 3 articles")
article_id = article1.id

# List and filter articles
all_articles = crud.list_articles(db, limit=10)
print(f"✅ Total articles: {len(all_articles)}")

papers = crud.list_articles(db, source_type="paper")
print(f"✅ Papers only: {len(papers)}")

top_articles = crud.get_top_articles_by_importance(db, limit=5)
print(f"✅ Top 5 articles by importance:")
for a in top_articles:
    print(f"   - {a.title[:50]}... (score: {a.importance_score})")

### 1.5 SentDigest & Feedback CRUD Operations

In [ ]:
# Create digest
digest = crud.create_digest(
    db=db,
    user_id=user_id,
    article_ids=[str(article1.id), str(article2.id)],
)
print(f"✅ Created digest: {digest.id}")
print(f"   Articles: {len(digest.article_ids)}")

digest_id = digest.id

# Create feedback
feedback1 = crud.create_feedback(
    db=db,
    user_id=user_id,
    article_id=article1.id,
    rating=5,
    comment="Excellent paper!",
)
print(f"✅ Created feedback: rating {feedback1.rating}")

# Get average rating
avg_rating = crud.get_article_average_rating(db, article1.id)
print(f"✅ Average rating: {avg_rating}")

### 1.6 Test Relationships

In [ ]:
# Test User → Preference, Digests, Feedbacks
user = crud.get_user_by_id(db, user_id)
print(f"User: {user.email}")
print(f"✅ Has preference: {user.preference is not None}")
print(f"✅ Digests: {len(user.digests)}")
print(f"✅ Feedbacks: {len(user.feedbacks)}")

# Test Article → Feedbacks
article = crud.get_article_by_id(db, article1.id)
print(f"\nArticle: {article.title}")
print(f"✅ Feedbacks: {len(article.feedbacks)}")

print("\n✅ All relationships working correctly!")

### ✅ Checkpoint 1 완료!

- Database connection
- User, UserPreference, CollectedArticle, SentDigest, Feedback CRUD
- Relationships (1:1, 1:N) 및 CASCADE 삭제
- Filtering, Sorting, Pagination

---

# Checkpoint 2: Scheduler & Scheduled Tasks

APScheduler를 사용한 스케줄러 설정 및 태스크 실행을 테스트합니다.

### 2.1 Scheduler Setup

In [ ]:
from app.scheduler.main import (
    start_scheduler,
    stop_scheduler,
    get_scheduler_status,
    trigger_job_manually,
    scheduler,
)

print("✅ Scheduler imported successfully!")

# Start scheduler
start_scheduler()

# Check status
status = get_scheduler_status()
print(f"\n✅ Scheduler running: {status['running']}")
print(f"   Timezone: {status['timezone']}")
print(f"   Jobs: {len(status['jobs'])}")

print("\nScheduled Jobs:")
for job in status['jobs']:
    print(f"\n  - {job['name']} (ID: {job['id']})")
    print(f"    Next run: {job['next_run_time']}")
    print(f"    Trigger: {job['trigger']}")

### 2.2 Test Data Collection Task

In [ ]:
from app.scheduler.tasks import collect_data_task

# Check initial count
initial_count = crud.count_articles(db)
print(f"Initial article count: {initial_count}")

# Run collection task
print("\nRunning data collection task...")
print("=" * 60)
collect_data_task()
print("=" * 60)

# Check after count
after_count = crud.count_articles(db)
collected = after_count - initial_count
print(f"\n✅ Data collection completed!")
print(f"   Articles collected: {collected}")
print(f"   Total articles: {after_count}")

### 2.3 Test Article Processing Task

In [ ]:
from app.scheduler.tasks import process_articles_task

# Check unprocessed articles
all_articles = crud.list_articles(db, limit=100)
unprocessed = [a for a in all_articles if not a.summary or a.importance_score is None]
print(f"Unprocessed articles: {len(unprocessed)}")

if len(unprocessed) > 0:
    print("\nRunning article processing task...")
    print("=" * 60)
    process_articles_task()
    print("=" * 60)
    print("\n✅ Article processing completed!")
    
    # Show processed articles
    processed_articles = crud.list_articles(db, limit=3)
    print("\nSample processed articles:")
    for i, article in enumerate(processed_articles, 1):
        print(f"\n{i}. {article.title[:60]}...")
        print(f"   Summary: {article.summary[:80] if article.summary else 'None'}...")
        print(f"   Importance: {article.importance_score}")
        print(f"   Category: {article.category}")
else:
    print("⚠️ No unprocessed articles found")

### 2.4 Test Scheduler Lifecycle

In [ ]:
import time

# Check status
status = get_scheduler_status()
print(f"Scheduler running: {status['running']}")

# Stop scheduler
print("\nStopping scheduler...")
stop_scheduler()
time.sleep(1)
status = get_scheduler_status()
print(f"✅ Scheduler running: {status['running']}")

# Start again
print("\nStarting scheduler again...")
start_scheduler()
time.sleep(1)
status = get_scheduler_status()
print(f"✅ Scheduler running: {status['running']}")
print(f"   Jobs: {len(status['jobs'])}")

### ✅ Checkpoint 2 완료!

- Scheduler 초기화 및 상태 확인
- 3개 scheduled jobs 등록 (collect_data, process_articles, send_digests)
- 개별 태스크 수동 실행
- Scheduler start/stop 라이프사이클

---

# Checkpoint 3: Full Pipeline Integration

전체 파이프라인 (수집 → 처리 → 저장 → 큐레이션 → 발송)을 End-to-end로 테스트합니다.

### 3.1 Create Integration Test User

In [ ]:
# Create integration test user
integration_user = crud.create_user(
    db=db,
    email="integration_test@example.com",
    name="Integration Test User"
)

crud.create_user_preference(
    db=db,
    user_id=integration_user.id,
    research_fields=["Machine Learning", "NLP", "Computer Vision"],
    keywords=["transformer", "attention", "LLM", "GPT", "BERT"],
    email_time="09:00",
    daily_limit=10,
    email_enabled=False,
)

print(f"✅ Created integration test user: {integration_user.email}")
print(f"   User ID: {integration_user.id}")

integration_user_id = integration_user.id

### 3.2 Run Full Pipeline

In [ ]:
print("Running Full Pipeline Integration Test\n")
print("=" * 60)

# 1. Data Collection
print("\n1️⃣ Data Collection...")
initial_count = crud.count_articles(db)
collect_data_task()
after_collect = crud.count_articles(db)
print(f"   ✅ Collected: {after_collect - initial_count} articles")

# 2. Article Processing
print("\n2️⃣ Article Processing...")
process_articles_task()
print(f"   ✅ Processing completed")

# 3. Database Statistics
print("\n3️⃣ Database Statistics")
total = crud.count_articles(db)
papers = crud.count_articles(db, source_type="paper")
news = crud.count_articles(db, source_type="news")
print(f"   Total: {total} | Papers: {papers} | News: {news}")

# 4. Top Articles
print("\n4️⃣ Top Articles by Importance")
top_articles = crud.get_top_articles_by_importance(db, limit=5)
for i, article in enumerate(top_articles, 1):
    print(f"   {i}. [{article.importance_score:.3f}] {article.title[:50]}...")

# 5. Create Digest
print("\n5️⃣ Digest Creation")
if len(top_articles) > 0:
    digest = crud.create_digest(
        db=db,
        user_id=integration_user_id,
        article_ids=[str(a.id) for a in top_articles],
    )
    print(f"   ✅ Created digest: {len(digest.article_ids)} articles")

print("\n" + "=" * 60)
print("✅ Full Pipeline Integration Test Completed!")

### 3.3 Verify Relationships & Data Integrity

In [ ]:
# Get user with all relationships
user = crud.get_user_by_id(db, integration_user_id)

print("User Relationships:")
print("=" * 40)
print(f"User: {user.email}")
print(f"✅ Has preference: {user.preference is not None}")
if user.preference:
    print(f"   Research fields: {user.preference.research_fields}")
    print(f"   Daily limit: {user.preference.daily_limit}")

print(f"\n✅ Digests: {len(user.digests)}")
for digest in user.digests:
    print(f"   - {digest.sent_at}: {len(digest.article_ids)} articles")

print(f"\n✅ Feedbacks: {len(user.feedbacks)}")

### 3.4 Pipeline Performance Metrics

In [ ]:
# Calculate metrics
all_articles = crud.list_articles(db, limit=100)
has_summary = sum(1 for a in all_articles if a.summary)
has_score = sum(1 for a in all_articles if a.importance_score is not None)
has_category = sum(1 for a in all_articles if a.category)
has_vector = sum(1 for a in all_articles if a.vector_id)

total = len(all_articles)

print("📈 Pipeline Performance Metrics")
print("=" * 40)
print(f"Total articles: {total}")
print(f"\nProcessing completion rates:")
if total > 0:
    print(f"  Summarization: {has_summary}/{total} ({has_summary/total*100:.1f}%)")
    print(f"  Importance scoring: {has_score}/{total} ({has_score/total*100:.1f}%)")
    print(f"  Categorization: {has_category}/{total} ({has_category/total*100:.1f}%)")
    print(f"  Embedding generation: {has_vector}/{total} ({has_vector/total*100:.1f}%)")
    
    if has_score > 0:
        avg_score = sum(a.importance_score for a in all_articles if a.importance_score) / has_score
        print(f"\nAverage importance score: {avg_score:.3f}")
else:
    print("  No articles to analyze")

### ✅ Checkpoint 3 완료!

- 데이터 수집 (arXiv + News)
- LLM 처리 (요약, 평가, 분류, 임베딩)
- 데이터베이스 저장 및 관계 검증
- 다이제스트 큐레이션
- 성능 메트릭 분석

---

# Checkpoint 4: Scheduler API & Monitoring

REST API를 통한 스케줄러 모니터링 및 제어를 테스트합니다.

**⚠️ 주의**: 이 섹션을 실행하기 전에 FastAPI 서버를 시작해야 합니다.

```bash
# 별도 터미널에서 실행
uvicorn src.app.api.main:app --reload --port 8000
```

### 4.1 Setup API Client

In [ ]:
import httpx
import json

# API base URL
BASE_URL = "http://localhost:8000/api/scheduler"

print("✅ API Client setup")
print(f"   Base URL: {BASE_URL}")

# Test if server is running
try:
    response = httpx.get("http://localhost:8000/health", timeout=2.0)
    if response.status_code == 200:
        print("✅ FastAPI server is running")
    else:
        print(f"⚠️  Server returned: {response.status_code}")
except Exception as e:
    print(f"❌ Could not connect to server: {e}")
    print("   Please start the FastAPI server first:")
    print("   uvicorn src.app.api.main:app --reload --port 8000")

### 4.2 Test GET /api/scheduler/status

In [ ]:
response = httpx.get(f"{BASE_URL}/status")

print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print("\n✅ GET /status successful\n")
    print(json.dumps(data, indent=2, ensure_ascii=False))
    
    print(f"\nScheduler Running: {data['running']}")
    print(f"Timezone: {data['timezone']}")
    print(f"Jobs: {len(data['jobs'])}")
else:
    print(f"❌ Error: {response.text}")

### 4.3 Test GET /api/scheduler/jobs

In [ ]:
response = httpx.get(f"{BASE_URL}/jobs")

print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    data = response.json()
    print("\n✅ GET /jobs successful\n")
    
    print(f"Total Jobs: {data['total']}\n")
    for i, job in enumerate(data['jobs'], 1):
        print(f"{i}. {job['name']}")
        print(f"   ID: {job['id']}")
        print(f"   Next Run: {job['next_run_time']}")
        print(f"   Trigger: {job['trigger']}")
        print()
else:
    print(f"❌ Error: {response.text}")

### 4.4 Test POST /api/scheduler/jobs/trigger (Optional)

⚠️ **WARNING**: This will actually run the job! Uncomment only if you want to test.

In [ ]:
# Uncomment to test job triggering
# job_id = 'collect_data'  # or 'process_articles' or 'send_digests'
# payload = {'job_id': job_id}
# response = httpx.post(f'{BASE_URL}/jobs/trigger', json=payload)
# 
# print(f"Status Code: {response.status_code}")
# if response.status_code == 200:
#     data = response.json()
#     print("\n✅ POST /jobs/trigger successful\n")
#     print(json.dumps(data, indent=2, ensure_ascii=False))
# else:
#     print(f"❌ Error: {response.text}")

print("⚠️  Job trigger test is commented out.")
print("   Uncomment the code above to test manual job triggering.")

### 4.5 Test Error Handling

In [ ]:
print("Testing Error Handling\n")
print("=" * 60)

# Test 1: Invalid job ID
print("\n1. Testing invalid job ID:")
response = httpx.post(
    f"{BASE_URL}/jobs/trigger",
    json={"job_id": "invalid_job_id"}
)
print(f"   Status: {response.status_code}")
if response.status_code == 404:
    print("   ✅ Correctly returned 404")

# Test 2: Invalid control action
print("\n2. Testing invalid control action:")
response = httpx.post(
    f"{BASE_URL}/control",
    json={"action": "invalid_action"}
)
print(f"   Status: {response.status_code}")
if response.status_code == 400:
    print("   ✅ Correctly returned 400")

# Test 3: Missing required field
print("\n3. Testing missing required field:")
response = httpx.post(
    f"{BASE_URL}/jobs/trigger",
    json={}
)
print(f"   Status: {response.status_code}")
if response.status_code == 422:
    print("   ✅ Correctly returned 422")

print("\n" + "=" * 60)
print("✅ Error handling tests completed!")

### 4.6 Verify API Documentation

In [ ]:
print("FastAPI Auto-Generated Documentation:\n")
print("=" * 60)
print("\n1. Swagger UI (Interactive):")
print("   http://localhost:8000/docs")
print("\n2. ReDoc (Alternative):")
print("   http://localhost:8000/redoc")
print("\n3. OpenAPI JSON Schema:")
print("   http://localhost:8000/openapi.json")
print("\n" + "=" * 60)

# Test if docs are accessible
try:
    response = httpx.get("http://localhost:8000/docs", follow_redirects=True, timeout=2.0)
    if response.status_code == 200:
        print("\n✅ API documentation is accessible!")
    else:
        print(f"\n⚠️  Documentation returned status code: {response.status_code}")
except Exception as e:
    print(f"\n❌ Could not access documentation: {e}")

### ✅ Checkpoint 4 완료!

- REST API 엔드포인트 구현 (4개)
  - GET /api/scheduler/status
  - GET /api/scheduler/jobs
  - POST /api/scheduler/jobs/trigger
  - POST /api/scheduler/control
- Pydantic 스키마 (7개)
- 에러 핸들링 (404, 400, 422)
- API 문서 자동 생성 (Swagger UI, ReDoc)

---

# Cleanup (Optional)

테스트 데이터를 정리합니다.

In [ ]:
# Uncomment to clean up test data
# crud.delete_user(db, user_id)  # Cascade deletes preferences, digests, feedbacks
# crud.delete_user(db, integration_user_id)
# crud.delete_article(db, article1.id)
# crud.delete_article(db, article2.id)
# crud.delete_article(db, article3.id)
# print("✅ Test data deleted")

# Stop scheduler
stop_scheduler()
print("✅ Scheduler stopped")

# Close database session
db.close()
print("✅ Database session closed")

---

# 🎉 Day 7 전체 테스트 완료!

## 테스트 완료 항목

### ✅ Checkpoint 1: Database Models & CRUD
- SQLAlchemy 모델 관계 설정 (ForeignKey, CASCADE)
- 40+ CRUD 연산 구현
- User, UserPreference, CollectedArticle, SentDigest, Feedback

### ✅ Checkpoint 2: Scheduler & Tasks
- APScheduler 설정 및 라이프사이클
- 3개 scheduled jobs (수집, 처리, 발송)
- Retry 로직 및 에러 핸들링

### ✅ Checkpoint 3: Full Pipeline Integration
- 데이터 수집 → LLM 처리 → 저장 → 큐레이션
- End-to-end 파이프라인 검증
- 성능 메트릭 분석

### ✅ Checkpoint 4: Scheduler API & Monitoring
- REST API 엔드포인트 (4개)
- Pydantic 스키마 (7개)
- 에러 핸들링 및 API 문서 자동 생성

## 시스템 상태

**✅ Production Ready** (Vector DB integration pending)

- Database: 5 models, 40+ CRUD operations, 6 relationships
- Scheduler: 3 jobs, KST timezone, graceful shutdown
- API: 4 REST endpoints, 7 Pydantic schemas
- Pipeline: 수집 → 처리 → 저장 → 큐레이션 → 발송 → 모니터링

## Next Steps

- Vector DB operations 완성
- Streamlit 프론트엔드 구현
- 실제 SMTP 이메일 발송 테스트
- API 인증 (JWT) 추가